In [1]:
#Testing Pyspark 
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()

'C:\\spark'

In [2]:
from pyspark import SparkConf, SparkContext                          #Importing the Spark
import collections                              

# I/ Exploratory Data Analysis (EDA) 

In [3]:
conf = SparkConf().setMaster("local").setAppName("RatingsHistogram") #Some confrigurations 
sc = SparkContext(conf = conf)

In [6]:
lines = sc.textFile("E:/Coding/0.Additional Courses/SparkCourse/ml-100k/u.data")
ratings = lines.map(lambda x: x.split()[2])             #Parse the data into different values 
result = ratings.countByValue()                         #Count of them for each one for us 

In [7]:
sortedResults = collections.OrderedDict(sorted(result.items()))
for key, value in sortedResults.items():
    print("%s %i" % (key, value))
                                      #Show the result and see what is going on 

1 6110
2 11370
3 27145
4 34174
5 21201


In [5]:
#Create the Resilient Distributed Dataset (RDD)
lines = sc.textFile("E:/Coding/0.Additional Courses/SparkCourse/ml-100k/u.data")   #When they input the information in, it is each line is each line of text, not exactly the same like dataframe  
movies = lines.map(lambda x: (int(x.split()[1]),1))             
                                    #Pull out nothing but a list of movie id 
                                    #Add 1 so that we can add them up together later 
movieCounts = movies.reduceByKey(lambda x,y : x+y)

In [6]:
#Sorting the customer by their spending 
sortedMovies = movieCounts.map(lambda x: (x[1], x[0])).sortByKey() 
                                        #Same function as: totalByCustomer.map(lambda x: (x[1], x[0])) = totalByCustomer.map(lambda (x,y):(y,x))

In [7]:
#Show the result and see what is going on 
results = sortedMovies.collect()
for result in results:
    print(result)      

(1, 1348)
(1, 1320)
(1, 1492)
(1, 1364)
(1, 1493)
(1, 830)
(1, 1498)
(1, 814)
(1, 1520)
(1, 711)
(1, 1373)
(1, 1309)
(1, 857)
(1, 1236)
(1, 1310)
(1, 1536)
(1, 1582)
(1, 1343)
(1, 1457)
(1, 1543)
(1, 599)
(1, 1458)
(1, 1561)
(1, 1533)
(1, 1565)
(1, 1563)
(1, 1156)
(1, 1505)
(1, 852)
(1, 1557)
(1, 1562)
(1, 1586)
(1, 1476)
(1, 1580)
(1, 1363)
(1, 1339)
(1, 1566)
(1, 1349)
(1, 1447)
(1, 1235)
(1, 1587)
(1, 677)
(1, 1571)
(1, 1575)
(1, 1510)
(1, 1579)
(1, 1603)
(1, 1616)
(1, 1526)
(1, 1596)
(1, 1453)
(1, 1461)
(1, 1559)
(1, 1507)
(1, 1593)
(1, 1576)
(1, 1525)
(1, 1569)
(1, 1568)
(1, 1340)
(1, 1619)
(1, 1601)
(1, 1583)
(1, 1624)
(1, 1651)
(1, 1414)
(1, 1486)
(1, 1614)
(1, 1570)
(1, 1599)
(1, 1649)
(1, 1572)
(1, 1653)
(1, 1452)
(1, 1595)
(1, 1548)
(1, 1655)
(1, 1654)
(1, 1482)
(1, 1657)
(1, 1650)
(1, 1660)
(1, 1661)
(1, 1515)
(1, 1621)
(1, 1632)
(1, 1618)
(1, 1647)
(1, 1581)
(1, 1584)
(1, 1669)
(1, 1613)
(1, 1130)
(1, 1663)
(1, 1634)
(1, 1606)
(1, 1329)
(1, 1494)
(1, 1673)
(1, 1633)
(1, 167

Movie ID=50 is the most popular movie with 583 times people watched  

In [8]:
#Create a dictionary to match the movie id to movie names
def loadMovieNames():
    movieNames = {}
    with open("E:/Coding/0.Additional Courses/SparkCourse/ml-100k/u.ITEM") as f:      #open the file and go to every road in that file 
        for line in f:
            fields = line.split('|')
            movieNames[int(fields[0])] = fields[1] #Extract movie ID and convert it into integer, use that as a key for our dictionary  
    return movieNames
                            #U have a key value and it will let u assign the key value to that key or retrive the value from the key 

In [9]:
#Call the load Movie name function which return a dictionary in Python that map the movieID to movie name and braodcast it to every node in our cluster 
nameDict = sc.broadcast(loadMovieNames())
                    #now we will using nameDict instead of the "loadMovieNames"

In [10]:
#Create the Resilient Distributed Dataset (RDD)
lines = sc.textFile("E:/Coding/0.Additional Courses/SparkCourse/ml-100k/u.data")   #When they input the information in, it is each line is each line of text, not exactly the same like dataframe  
movies = lines.map(lambda x: (int(x.split()[1]),1))             
                                    #Pull out nothing but a list of movie id 
                                    #Add 1 so that we can add them up together later 
movieCounts = movies.reduceByKey(lambda x,y : x+y)

In [11]:
#Sorting the customer by the number of times a movie is watched
sortedMovies = movieCounts.map(lambda x: (x[1], x[0])).sortByKey() 
                                        #Same function as: totalByCustomer.map(lambda x: (x[1], x[0])) = totalByCustomer.map(lambda (x,y):(y,x))

In [12]:
#Lookup the sorted movie and replace them with name 
sortedMoviesWithNames = sortedMovies.map(lambda countMovie : (nameDict.value[countMovie[1]], countMovie[0]))
                                        #(lambda countMovie : (nameDict.value[countMovie[1]], countMovie[0]))  =  map(lambda(count,movie):(nameDict.value[movie],count))
                                        # map takes in each key(count)-value(movie) pair. And they will replace it with the new key-value piar where we're using the name dict object was boracasted to every node
                                        #we call value on it in order to retrive the origincal dictionary object and use that to look up the name for that given movie idea 
                                        #So out mapper is taking a  key value par of count and movie ID and transforming that into a movie name and account using nameDict broadcast
                
                                        #we can only do this within a mapper only because we broadcast that to every node ahead of time. 
                                        #If we were not using a boroacast varible it was still work but we would end up transmitting teh entire dictionary across the wire possibly more times than it would need to that point  

In [13]:
#Show the result and see what is going on 
results = sortedMoviesWithNames.collect()
for result in results:
    print(result)          

('Every Other Weekend (1990)', 1)
('Homage (1995)', 1)
('Window to Paris (1994)', 1)
('Bird of Prey (1996)', 1)
('Modern Affair, A (1995)', 1)
('Power 98 (1995)', 1)
('Farmer & Chase (1995)', 1)
('Great Day in Harlem, A (1994)', 1)
('Fear, The (1995)', 1)
('Substance of Fire, The (1996)', 1)
('Good Morning (1971)', 1)
('Very Natural Thing, A (1974)', 1)
('Paris Was a Woman (1995)', 1)
('Other Voices, Other Rooms (1997)', 1)
('Walk in the Sun, A (1945)', 1)
('Aiqing wansui (1994)', 1)
('T-Men (1947)', 1)
('Lotto Land (1995)', 1)
('Love Is All There Is (1996)', 1)
('Johns (1996)', 1)
('Police Story 4: Project S (Chao ji ji hua) (1993)', 1)
('Damsel in Distress, A (1937)', 1)
('Tigrero: A Film That Was Never Made (1994)', 1)
("I Don't Want to Talk About It (De eso no se habla) (1993)", 1)
('Daens (1992)', 1)
('Promise, The (Versprechen, Das) (1994)', 1)
('Cyclo (1995)', 1)
('Killer: A Journal of Murder (1995)', 1)
('Bloody Child, The (1996)', 1)
('Yankee Zulu (1994)', 1)
("Eye of Vichy, T

StarWar(1997) is the most popular movie with 583 times people watched  

# II/ Movie Recommendations and potential model optimization methods

# How we plan to do it  (This is just one way to do it)

- Find every pair of movies that were watched by the same person 
- Measure the similarity of their ratings across all users who watched both 
- Sort by movie, then by similarity strength 

# How we are going to code it 


- Map input ratings to (userID,(movieID,rating))
- Find every movie pair rated to the same user 
        This can be done with a "self-join" operation
        At this point we have (userID,((movieID1,rating1),(movieID2,rating2)))
- Filter out duplicate pairs 
- Make the movie pairs the key 
        map to ((movieID1,movieID2), (rating1,rating2))    /// movieID are keys, rating are values  
- groupByKey() to get every rating pair found for each movie pair
- Compute similarity between ratigs for each movie in the pair 
- Sort, save & display results 

In [15]:
#Importing the Spark
from pyspark import SparkConf, SparkContext                          
import sys 
from math import sqrt 

In [16]:
#Writing the Functions

def loadMovieNames():
    movieNames = {}
    with open("E:/Coding/0.Additional Courses/SparkCourse/ml-100k/u.item", encoding='ascii', errors='ignore') as f:
        for line in f:
            fields = line.split('|')
            movieNames[int(fields[0])] = fields[1]
    return movieNames
                                        #Return the Python dictionary, match movieID to readable name


    
#Python 3 doesn't let you pass around unpacked tuples,
#so we explicitly extract the ratings now.
def filterDuplicates( userRatings ):
    ratings = userRatings[1]
    (movie1, rating1) = ratings[0]
    (movie2, rating2) = ratings[1]
    return movie1 < movie2
                                        #Filter duplicates by returning that movie1 < movie2, meaning we dispose movie1=movie2 

def makePairs( userRatings ):
    ratings = userRatings[1]
    (movie1, rating1) = ratings[0]
    (movie2, rating2) = ratings[1]
    return ((movie1, movie2), (rating1, rating2))
                                        #Make the pairs of movies ratings taht we will use below 

def computeCosineSimilarity(ratingPairs):
    numPairs = 0
    sum_xx = sum_yy = sum_xy = 0
    for ratingX, ratingY in ratingPairs:
        sum_xx += ratingX * ratingX
        sum_yy += ratingY * ratingY
        sum_xy += ratingX * ratingY
        numPairs += 1

    numerator = sum_xy
    denominator = sqrt(sum_xx) * sqrt(sum_yy)

    score = 0
    if (denominator):
        score = (numerator / (float(denominator)))

    return (score, numPairs)

In [17]:
#Load the data 
nameDict = loadMovieNames()
                            #use the function loadMovieNames()

data = sc.textFile("E:/Coding/0.Additional Courses/SparkCourse/ml-100k/u.data")

ALL THE OPERATIONAL CODES

In [18]:
ratings = data.map(lambda l: l.split()).map(lambda l: (int(l[0]), (int(l[1]), float(l[2]))))
                                                #data.map(lambda l: l.split()) - splitting the data on whitespace
                                                #map it again to the format we want 
                                                #(Key) userID 
                                                #(Value) movieID, rating
joinedRatings = ratings.join(ratings)       
                                # Emit every movie rated together by the same user.
                                # Self-join to find every combination.
                                #This is the most heavy part here

==> At this point our RDD consists of userID => ((movieID, rating), (movieID, rating))

In [19]:
uniqueJoinedRatings = joinedRatings.filter(filterDuplicates)
                                # Filter out duplicate pairs
moviePairs = uniqueJoinedRatings.map(makePairs)
                                # Now key by (movie1, movie2) pairs.
moviePairRatings = moviePairs.groupByKey()
                                #After we turn the mpvie pairs into key, we group them together 
                                # We now have (movie1, movie2) => (rating1, rating2)
                                # Now collect all ratings for each movie pair and compute similarity
moviePairSimilarities = moviePairRatings.mapValues(computeCosineSimilarity).cache()
                                # We now have (Key)(movie1, movie2) = > (value)(rating1, rating2), (rating1, rating2) ...
                                # Can now compute similarities.
                                #cache() is very important here so we need to catch the results to reuse it again for other analysis more than 1, if not, we have to do everything from the beginning 

In [20]:
# Save the results if desired
#moviePairSimilarities.sortByKey()
#moviePairSimilarities.saveAsTextFile("movie-sims")

In [21]:
# Extract similarities for the movie we care about that are "good".
print("\nLoading movie names...")
if (len(sys.argv) > 1):

    scoreThreshold = 0.97
    coOccurenceThreshold = 50

    movieID = int(sys.argv[1])        #This is the movieID that we will be looking for 

    # Filter for movies with this sim that are "good" as defined by
    # our quality thresholds above
    filteredResults = moviePairSimilarities.filter(lambda pairSim: \
        (pairSim[0][0] == movieID or pairSim[0][1] == movieID) \
        and pairSim[1][0] > scoreThreshold and pairSim[1][1] > coOccurenceThreshold)
   
                                                                    #Find the similar movies to Starwar in the pair             
                                                                    #Want to find the pir with the score higher than the score threshold

    # Sort by quality score.
    results = filteredResults.map(lambda pairSim: (pairSim[1], pairSim[0])).sortByKey(ascending = False).take(10)
                                                                                        #Sort the movies by score that they have 
                                                                                        #1/ Flip the key & value to find the to have the score goes first
                                                                                        #2/ Sort by key with the accessding order and take the first 10 

    print("Top 10 similar movies for " + nameDict[movieID])
    for result in results:
        (sim, pair) = result
        # Display the similarity result that isn't the movie we're looking at
        similarMovieID = pair[0]
        if (similarMovieID == movieID):
            similarMovieID = pair[1]
        print(nameDict[similarMovieID] + "\tscore: " + str(sim[0]) + "\tstrength: " + str(sim[1]))



Loading movie names...


ValueError: invalid literal for int() with base 10: '-f'

It is noted that the model we are using implemented the ‘Coside metrics’. We also set the threshold for the minimum co-raters or minimum scores as 0.95 and the Occurrence threshold as 50  . Unlike the other project, this cannot be done with the script on Jupyter Notebook but rather Anaconda prompt. In the Anaconda prompt from your start menu and run the command ' spark-submit Movies-Similarities.py 50'. Make sure to save the file in .py rather than .ipynb format and run your code on the right drive so that it will run smoothly. 

-->  Change the thres holds to get the comparison results as stated in the paper 

# III/ Movie Recommendations on the Cloud Cluster (Amazon Web Service-EC2) 

In [ ]:
#Set up the context
conf = SparkConf().
                                    #SparkConf is empty rather than having something behind it because I am gonna pass it on the command line 
                                    #Take advantage if sime of the pre-confriguated stuff on EMR elastic mapo reduce that will automatically tell Spark to run on top of Hadoop Yarn 
sc = SparkContext(conf = conf)

In [ ]:
#To run on EMR successfully + output results for Star Wars:
#aws s3 cp s3://sundog-spark/MovieSimilarities1M.py ./
#aws s3 sp c3://sundog-spark/ml-1m/movies.dat ./
#spark-submit --executor-memory 1g MovieSimilarities1M.py 260

In [ ]:
#Writing the Functions

def loadMovieNames():
    movieNames = {}
    with open("E:/Coding/0.Additional Courses/SparkCourse/ml-1m/movies.dat") as f:
        for line in f:
            fields = line.split("::")
            movieNames[int(fields[0])] = fields[1].decode('ascii', 'ignore')
    return movieNames
                                        #Return the Python dictionary, match movieID to readable name


    
#Python 3 doesn't let you pass around unpacked tuples,
#so we explicitly extract the ratings now.
def filterDuplicates( userRatings ):
    ratings = userRatings[1]
    (movie1, rating1) = ratings[0]
    (movie2, rating2) = ratings[1]
    return movie1 < movie2
                                        #Filter duplicates by returning that movie1 < movie2, meaning we dispose movie1=movie2 

def makePairs( userRatings ):
    ratings = userRatings[1]
    (movie1, rating1) = ratings[0]
    (movie2, rating2) = ratings[1]
    return ((movie1, movie2), (rating1, rating2))
                                        #Make the pairs of movies ratings taht we will use below 

def computeCosineSimilarity(ratingPairs):
    numPairs = 0
    sum_xx = sum_yy = sum_xy = 0
    for ratingX, ratingY in ratingPairs:
        sum_xx += ratingX * ratingX
        sum_yy += ratingY * ratingY
        sum_xy += ratingX * ratingY
        numPairs += 1

    numerator = sum_xy
    denominator = sqrt(sum_xx) * sqrt(sum_yy)

    score = 0
    if (denominator):
        score = (numerator / (float(denominator)))

    return (score, numPairs)

In [ ]:
#Load the data 
nameDict = loadMovieNames()
                            #use the function loadMovieNames()

data = sc.textFile("s3n://sundog-spark/ml-1m/ratings.dat")
                            #Load the data from s3n instead of the normal log on the Amazon simple stories service 
                            #The dataset is stored in the cloud system itself rather than the actual file that we save on our computer. (Sundog-Spark ) 

ALL THE OPERATIONAL CODES

In [ ]:
# Map ratings to key / value pairs: user ID => movie ID, rating
ratings = data.map(lambda l: l.split("::")).map(lambda l: (int(l[0]), (int(l[1]), float(l[2]))))
                                                #data.map(lambda l: l.split()) - splitting the data on whitespace
                                                #map it again to the format we want 
                                                #(Key) userID 
                                                #(Value) movieID, rating
                                                #The format is changed a little bit as well from '_' to '::'
ratingsPartitioned = ratings.partitionBy(100)
joinedRatings = ratingsPartitioned.join(ratingsPartitioned)      
                                # Emit every movie rated together by the same user.
                                # Self-join to find every combination.
                                #This is the most heavy part here   

==> At this point our RDD consists of userID => ((movieID, rating), (movieID, rating))

In [ ]:
uniqueJoinedRatings = joinedRatings.filter(filterDuplicates)
                                # Filter out duplicate pairs
moviePairs = uniqueJoinedRatings.map(makePairs).partitionBy(100)
                                # Now key by (movie1, movie2) pairs.
moviePairRatings = moviePairs.groupByKey()
                                #After we turn the mpvie pairs into key, we group them together 
                                # We now have (movie1, movie2) => (rating1, rating2)
                                # Now collect all ratings for each movie pair and compute similarity
moviePairSimilarities = moviePairRatings.mapValues(computeCosineSimilarity).persist()
                                # We now have (Key)(movie1, movie2) = > (value)(rating1, rating2), (rating1, rating2) ...
                                # Can now compute similarities.
                                #cache() is very important here so we need to catch the results to reuse it again for other analysis more than 1, if not, we have to do everything from the beginning 
                                

In [ ]:
# Save the results if desired
#moviePairSimilarities.sortByKey()
#moviePairSimilarities.saveAsTextFile("movie-sims")

In [ ]:
# Extract similarities for the movie we care about that are "good".
print("\nLoading movie names...")
if (len(sys.argv) > 1):

    scoreThreshold = 0.97
    coOccurenceThreshold = 50

    movieID = int(sys.argv[1])        #This is the movieID that we will be looking for 

    # Filter for movies with this sim that are "good" as defined by
    # our quality thresholds above
    filteredResults = moviePairSimilarities.filter(lambda((pair,sim)): \
        (pair[0] == movieID or pair[1] == movieID) \
        and sim[0] > scoreThreshold and sim[1] > coOccurenceThreshold)
   
                                                                    #Find the similar movies to Starwar in the pair             
                                                                    #Want to find the pir with the score higher than the score threshold

    # Sort by quality score.
    results = filteredResults.map(lambda((pair,sim)): (sim, pair)).sortByKey(ascending = False).take(10)
                                                                                        #Sort the movies by score that they have 
                                                                                        #1/ Flip the key & value to find the to have the score goes first
                                                                                        #2/ Sort by key with the accessding order and take the first 10 

    print("Top 10 similar movies for " + nameDict[movieID])
    for result in results:
        (sim, pair) = result
        # Display the similarity result that isn't the movie we're looking at
        similarMovieID = pair[0]
        if (similarMovieID == movieID):
            similarMovieID = pair[1]
        print(nameDict[similarMovieID] + "\tscore: " + str(sim[0]) + "\tstrength: " + str(sim[1]))


# IV/ Movie Recommendations system using Spark’s Machine Learning Library (MLlib)

In [ ]:
#Importing the Spark
import sys
from pyspark.mllib.recommendation import ALS, Rating 

 Added 3 more fake reviews in the u.DATA to test our model  

In [ ]:
#Create the MovieNames function 
def loadMovieNames():
    movieNames = {}
    with open("E:/Coding/0.Additional Courses/SparkCourse/ml-100k/u.ITEM", encoding='ascii', errors="ignore") as f:
        for line in f:
            fields = line.split('|')
            movieNames[int(fields[0])] = fields[1]
    return movieNames

In [ ]:
conf = SparkConf().setMaster("local[*]").setAppName("MovieRecommendationsALS")
sc = SparkContext(conf = conf)
sc.setCheckpointDir('checkpoint')

In [ ]:
print("\nLoading movie names...")
nameDict = loadMovieNames()

data = sc.textFile("E:/Coding/0.Additional Courses/SparkCourse/ml-100k/u.data")

ratings = data.map(lambda l: l.split()).map(lambda l: Rating(int(l[0]), int(l[1]), float(l[2]))).cache()

In [ ]:
# Build the recommendation model using Alternating Least Squares
print("\nTraining recommendation model...")
rank = 10
numIterations = 20
model = ALS.train(ratings, rank, numIterations)
                                #Train based on the 100k movie rank ratings 

In [ ]:
userID = int(sys.argv[1])

print("\nRatings for user ID " + str(userID) + ":")
userRatings = ratings.filter(lambda l: l[0] == userID)
for rating in userRatings.collect():
    print (nameDict[int(rating[1])] + ": " + str(rating[2]))

print("\nTop 10 recommendations:")
recommendations = model.recommendProducts(userID, 10)
for recommendation in recommendations:
    print (nameDict[int(recommendation[1])] + \
        " score " + str(recommendation[2]))

# Use the Anaconda Promt to run the code 

# V/ Simplify the code with SparkSQL and DataFrames

In [29]:
from pyspark.sql import SparkSession, Row, functions

In [30]:
#Create the function for to Load MovieNames 
def loadMovieNames():
    movieNames = {}
    with open("E:/Coding/0.Additional Courses/SparkCourse/ml-100k/u.ITEM") as f:
        for line in f:
            fields = line.split('|')
            movieNames[int(fields[0])] = fields[1]
    return movieNames

In [31]:
# Create a SparkSession (the config bit is only for Windows!)
spark = SparkSession.builder.config("spark.sql.warehouse.dir", "file:///C:/temp").appName("PopularMovies").getOrCreate()

In [32]:
# Load up our movie ID -> name dictionary
nameDict = loadMovieNames()
                        #To print out readable results

In [33]:
# Get the raw data
lines = spark.sparkContext.textFile("E:/Coding/0.Additional Courses/SparkCourse/ml-100k/u.data")

# Convert it to a RDD of Row objects
movies = lines.map(lambda x: Row(movieID =int(x.split()[1])))
                                #Take each line of the data to create each row object that consists of the name movie ID
                                #Now we have the RDD name 'movie' that consists of row object, that each row consits of a movieID 

# Convert that to a DataFrame
movieDataset = spark.createDataFrame(movies)

In [34]:
# Some SQL-style magic to sort all movies by popularity in one line! and print out the result 
topMovieIDs = movieDataset.groupBy("movieID").count().orderBy("count", ascending=False).cache()
topMovieIDs.show()

+-------+-----+
|movieID|count|
+-------+-----+
|     50|  584|
|    258|  509|
|    100|  508|
|    181|  507|
|    294|  485|
|    286|  481|
|    288|  478|
|      1|  452|
|    300|  431|
|    121|  429|
|    174|  420|
|    127|  413|
|     56|  394|
|      7|  392|
|     98|  390|
|    237|  384|
|    117|  378|
|    172|  368|
|    222|  365|
|    313|  350|
+-------+-----+
only showing top 20 rows



In [35]:
# Grab the top 10
top10 = topMovieIDs.take(10)

# Print the results
print("\n")
for result in top10:
    # Each row has movieID, count as above.
    print("%s: %d" % (nameDict[result[0]], result[1]))
                                        #To match the result with human readable movie name

# Stop the session
spark.stop()



Star Wars (1977): 584
Contact (1997): 509
Fargo (1996): 508
Return of the Jedi (1983): 507
Liar Liar (1997): 485
English Patient, The (1996): 481
Scream (1996): 478
Toy Story (1995): 452
Air Force One (1997): 431
Independence Day (ID4) (1996): 429
